In [1]:
import matplotlib.pyplot as plt
import tqdm
import pandas as pd
from sklearn.model_selection import train_test_split
path = r"D:\datasets\CarSIM"
data = pd.read_csv(path+r"\Data.csv")


In [2]:
data.columns

Index(['PicID', 'Vertical', 'Horizontal', 'Velocity'], dtype='object')

In [3]:
img = plt.imread(path+"\\PIC\\"+data['PicID'][0]+".png")

In [4]:
img[:,:,:3].shape

(256, 256, 3)

In [67]:
imgs = []
vels = []
labels = []
for i, j, v, h in tqdm.tqdm(zip(data['PicID'], data['Velocity'], data['Vertical'], data['Horizontal'])):
    try:
        img = plt.imread(path+"\\PIC\\"+i+".png")
        imgs.append(img[:,:,:3])
        vels.append(j)

        labels.append(((v+1), (h+1)))
    except:
        pass



2050it [00:14, 140.11it/s]


In [68]:
print(imgs[0].shape)

(256, 256, 3)


In [69]:
len(labels)

2050

In [70]:
len(imgs)

2050

In [71]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, Dense, Flatten, MaxPooling2D, Dropout, Concatenate

In [151]:
class PolicyNetwork(Model):
    def __init__(self):
        super(PolicyNetwork, self).__init__()

        self.cnn = Sequential([
            Conv2D(12, 5, activation='relu'),
            Dropout(0.4),
            MaxPooling2D(3),
            Conv2D(25, 3, activation='relu'),
            Dropout(0.4),
            MaxPooling2D(3),
            Flatten(),
        ])
        
        self.concat = Concatenate(axis=1)
        self.concat2= Concatenate()
        self.dense = Sequential([
            Dense(12, activation='relu'),
            Dropout(0.2),
            Dense(12, activation='relu'),
            
        ])
        self.vertical = Dense(3, activation='softmax')
        self.horizontal = Dense(3, activation='softmax')

    def call(self, inps):
        img, vel = inps
        imgx = self.cnn(img)
        x = self.concat([imgx, vel])
        x = self.dense(x)
        v = self.vertical(x)
        h = self.horizontal(x)

        #v = tf.cast (tf.reshape(tf.argmax(v, axis=-1),(-1, 1)), tf.float32)
        
        #h = tf.cast (tf.reshape(tf.argmax(h, axis=-1),(-1, 1)) , tf.float32)
        return tf.reshape(Concatenate(axis=1)([v, h]), (-1, 2, 3))
        
    def get_action(self, img, vel):
        imgx = self.cnn(img)
        x = self.concat([imgx, vel])
        x = self.dense(x)
        v = self.vertical(x)
        h = self.horizontal(x)

        v = tf.argmax(v, axis=-1).numpy()[0] -1
        h = tf.argmax(h, axis=-1).numpy()[0] -1
        return np.array([v, h], dtype=np.float32)
model = PolicyNetwork()
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])

In [87]:
imgsx = np.array(imgs[20:], dtype=np.float32)
velsx = np.array(vels[20:], dtype=np.float32).reshape(-1, 1)
print(imgsx.shape)
print(velsx.shape)

(2030, 256, 256, 3)
(2030, 1)


In [85]:
y = np.array(labels[20:], dtype=np.float32)

In [153]:
model.fit((imgsx, velsx), y,epochs=8)

Epoch 1/15
64/64 [==============================] - 6s 100ms/step - loss: 0.6036 - acc: 0.7815
Epoch 2/15
64/64 [==============================] - 6s 100ms/step - loss: 0.5733 - acc: 0.7823
Epoch 3/15
64/64 [==============================] - 6s 101ms/step - loss: 0.5384 - acc: 0.7909
Epoch 4/15
64/64 [==============================] - 6s 101ms/step - loss: 0.5152 - acc: 0.7983
Epoch 5/15
64/64 [==============================] - 6s 100ms/step - loss: 0.5146 - acc: 0.8037
Epoch 6/15
64/64 [==============================] - 6s 99ms/step - loss: 0.5041 - acc: 0.8017
Epoch 7/15
64/64 [==============================] - 6s 100ms/step - loss: 0.4922 - acc: 0.8037
Epoch 8/15
64/64 [==============================] - 6s 101ms/step - loss: 0.4742 - acc: 0.8116
Epoch 9/15
64/64 [==============================] - 6s 100ms/step - loss: 0.4719 - acc: 0.8099
Epoch 10/15
64/64 [==============================] - 6s 94ms/step - loss: 0.4585 - acc: 0.8163
Epoch 11/15
64/64 [==============================] 

KeyboardInterrupt: 

In [152]:
model.fit((imgsx, velsx), y)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set

In [156]:
model.save_weights("Weights/model")

In [33]:
y.shape

(2030, 2)

In [154]:
v = model.get_action(imgsx[100:101], velsx[100:101])

In [155]:
v

array([1., 0.], dtype=float32)

0

<tf.Tensor: shape=(2030, 2, 3), dtype=float32, numpy=
array([[[0.32660303, 0.36742574, 0.3059713 ],
        [0.3284108 , 0.31261933, 0.3589699 ]],

       [[0.32543692, 0.3692265 , 0.3053366 ],
        [0.3280562 , 0.31252268, 0.35942116]],

       [[0.32386285, 0.3726084 , 0.30352876],
        [0.32800397, 0.31193453, 0.3600615 ]],

       ...,

       [[0.30572492, 0.4071887 , 0.28708637],
        [0.32824448, 0.31444642, 0.35730907]],

       [[0.30437154, 0.41053435, 0.28509405],
        [0.32751516, 0.3145497 , 0.35793516]],

       [[0.3024001 , 0.4129769 , 0.28462306],
        [0.32731256, 0.3120896 , 0.36059782]]], dtype=float32)>